# Libraries

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
from matplotlib.ticker import LinearLocator,MultipleLocator, AutoMinorLocator
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['text.usetex'] = True
rcParams['font.size'] = 20

In [2]:
PI=np.pi

# Functions

### DOS: $~~~~~\Lambda(E)=\frac{2}{\pi}\frac{E}{\sqrt{E^2-\Delta^2}\sqrt{J^2+\Delta^2-E^2}}~\Theta(E-\Delta)\Theta(\sqrt{J^2+\Delta^2}-E)$

In [4]:
def DOS(E,gap):
    return (2/PI)*E/(np.sqrt(np.abs(E**2-gap**2))*np.sqrt(np.abs(1+gap**2-E**2)))
def DOS2(E,Ep,gap):
    return (4.0/PI**2)*(E/(np.sqrt(np.abs(E**2-gap**2))*np.sqrt(np.abs(1+gap**2-E**2))))*(Ep/(np.sqrt(np.abs(Ep**2-gap**2))*np.sqrt(np.abs(1+gap**2-Ep**2))))

### Photonic spectral function: $~~~~~\mathrm{Im}D^R_{0}(\omega)=-\frac{1}{4\pi}\left[\frac{\gamma_B}{(\omega-\nu_0)^2+\gamma_B^2}-\frac{\gamma_B}{(\omega+\nu_0)^2+\gamma_B^2}\right]$

In [5]:
def ImDR0(w,nu0,gammaB):
    return -(1.0/(4.0*PI))*((gammaB/((w-nu0)**2+gammaB**2))-(gammaB/((w+nu0)**2+gammaB**2)))

### Distribution factor: $~~~~~H_0(\omega,\omega')=-4\left[n_0(\omega)-n_0(\omega')\right]\left[N_{T_{cav}}(\omega-\omega')-N_{T_{cry}}(\omega-\omega')\right]$

In [6]:
def H0(w,wp,Tcry,Tcav):
    return (np.tanh(wp/(2*Tcry))- np.tanh(w/(2*Tcry)))*((1.0/np.tanh((wp-w)/(2*Tcav)))-(1.0/np.tanh((wp-w)/(2*Tcry))))

### calculates $-2\int dE\Lambda(E)\frac{\delta n(E)}{E}$

In [1]:
#SC
def num_integ_SC(E,Ep,gap,Tcry,Tcav,gammaB,nu0):
    return (DOS2(E,Ep,gap)/E)*((1+(gap**2)/(E*Ep))*ImDR0(Ep-E,nu0,gammaB)*H0(E,Ep,Tcry,Tcav)-(1-(gap**2)/(E*Ep))*ImDR0(Ep+E,nu0,gammaB)*H0(-E,Ep,Tcry,Tcav))
def numSC(eps,gap,Tcry,Tcav,gammaB,nu0):
    return sp.integrate.dblquad(num_integ_SC,gap,np.sqrt(1+gap**2),gap,lambda x: x,args=(gap,Tcry,Tcav,gammaB,nu0),epsabs=eps)[0]+sp.integrate.dblquad(num_integ_SC,gap,np.sqrt(1+gap**2),lambda x: x,np.sqrt(1+gap**2),args=(gap,Tcry,Tcav,gammaB,nu0),epsabs=eps)[0]
#CDW
def num_integ_CDW(E,Ep,gap,Tcry,Tcav,gammaB,nu0):
    return (DOS2(E,Ep,gap)/E)*((1-(gap**2)/(E*Ep))*ImDR0(Ep-E,nu0,gammaB)*H0(E,Ep,Tcry,Tcav)-(1+(gap**2)/(E*Ep))*ImDR0(Ep+E,nu0,gammaB)*H0(-E,Ep,Tcry,Tcav))
def numCDW(eps,gap,Tcry,Tcav,gammaB,nu0):
    return sp.integrate.dblquad(num_integ_CDW,gap,np.sqrt(1+gap**2),gap,lambda x: x,args=(gap,Tcry,Tcav,gammaB,nu0),epsabs=eps)[0]+sp.integrate.dblquad(num_integ_CDW,gap,np.sqrt(1+gap**2),lambda x: x,np.sqrt(1+gap**2),args=(gap,Tcry,Tcav,gammaB,nu0),epsabs=eps)[0]

#### calculates $\int dE\tilde{\nu}(E)\frac{\delta F(E)}{E}$

In [8]:
# The Numerataors
#SC
def num_integ_SC(E,Ep,gap,Tcry,Tcav,gammaB,nu0):
    return (DOS2(E,Ep,gap)/E)*((1+(gap**2)/(E*Ep))*ImDR0(Ep-E,nu0,gammaB)*H0(E,Ep,Tcry,Tcav)-(1-(gap**2)/(E*Ep))*ImDR0(Ep+E,nu0,gammaB)*H0(-E,Ep,Tcry,Tcav))
def numSC(eps,gap,Tcry,Tcav,gammaB,nu0):
    return sp.integrate.dblquad(num_integ_SC,gap,np.sqrt(1+gap**2),gap,lambda x: x,args=(gap,Tcry,Tcav,gammaB,nu0),epsabs=eps)[0]+sp.integrate.dblquad(num_integ_SC,gap,np.sqrt(1+gap**2),lambda x: x,np.sqrt(1+gap**2),args=(gap,Tcry,Tcav,gammaB,nu0),epsabs=eps)[0]
#CDW
def num_integ_CDW(E,Ep,gap,Tcry,Tcav,gammaB,nu0):
    return (DOS2(E,Ep,gap)/E)*((1-(gap**2)/(E*Ep))*ImDR0(Ep-E,nu0,gammaB)*H0(E,Ep,Tcry,Tcav)-(1+(gap**2)/(E*Ep))*ImDR0(Ep+E,nu0,gammaB)*H0(-E,Ep,Tcry,Tcav))
def numCDW(eps,gap,Tcry,Tcav,gammaB,nu0):
    return sp.integrate.dblquad(num_integ_CDW,gap,np.sqrt(1+gap**2),gap,lambda x: x,args=(gap,Tcry,Tcav,gammaB,nu0),epsabs=eps)[0]+sp.integrate.dblquad(num_integ_CDW,gap,np.sqrt(1+gap**2),lambda x: x,np.sqrt(1+gap**2),args=(gap,Tcry,Tcav,gammaB,nu0),epsabs=eps)[0]

In [9]:
#SC
def delgapSC(eps,g0,gamma,gap,Tcry,Tcav,gammaB,nu0):
    return -(g0**2/gamma)*numSC(eps,gap,Tcry,Tcav,gammaB,nu0)/deN(gap,Tcry)

In [10]:
#CDW
def delgapCDW(eps,g0,gamma,gap,Tcry,Tcav,gammaB,nu0):
    return -(g0**2/gamma)*numCDW(eps,gap,Tcry,Tcav,gammaB,nu0)/deN(gap,Tcry)

### Thermal gap equation

In [11]:
#T=0
def gapeq0_integ(E,gap):
    return DOS(E,gap)/E
def gapeq0(gap,kap):
    return sp.integrate.quad(gapeq0_integ,gap,np.sqrt(1+gap**2),args=(gap))[0]-1.0/kap
#T>0
def gap_eq_integ(E,gap,T):
    return (DOS(E,gap)/E)*np.tanh(E/(2*T))
def gap_eq(gap,T,kap):
    return sp.integrate.quad(gap_eq_integ,gap,np.sqrt(1+gap**2),args=(gap,T))[0]-1.0/kap
def gap_eq_inv(T,gap,kap):
    return sp.integrate.quad(gap_eq_integ,gap,np.sqrt(1+gap**2),args=(gap,T))[0]-1.0/kap

### Non-thermal gap equation 

In [12]:
#SC
def gap_nth_SC(gap,Tcry,kap,g0,gamma,gammaB,Tcav,nu0,eps):
    return gap_eq(gap,Tcry,kap)-(g0**2/gamma)*numSC(eps,gap,Tcry,Tcav,gammaB,nu0)
def gap_nth_SC_inv(Tcry,gap,kap,g0,gamma,gammaB,Tcav,nu0,eps):
    return gap_eq(gap,Tcry,kap)-(g0**2/gamma)*numSC(eps,gap,Tcry,Tcav,gammaB,nu0)
#CDW
def gap_nth_CDW(gap,Tcry,kap,g0,gamma,gammaB,Tcav,nu0,eps):
    return gap_eq(gap,Tcry,kap)-(g0**2/gamma)*numCDW(eps,gap,Tcry,Tcav,gammaB,nu0)
def gap_nth_CDW_inv(Tcry,gap,kap,g0,gamma,gammaB,Tcav,nu0,eps):
    return gap_eq(gap,Tcry,kap)-(g0**2/gamma)*numCDW(eps,gap,Tcry,Tcav,gammaB,nu0)

# Calculations (generate data)

In [13]:
NGAP=31
GAPlist=np.array([i*0.005 for i in range(0,NGAP)])
TEQlist=np.zeros([NGAP])
TSClist=np.zeros([NGAP])
TCDWlist=np.zeros([NGAP])

In [14]:
KAP=0.5
TCAV=0.20
G0=0.1
GAMMA=0.01
GAMMAB=0.1
NU0=0.01
EPS=0.00001
for i in range(0,NGAP):
    TEQlist[i]=sp.optimize.fsolve(gap_eq_inv,0.1,args=(GAPlist[i],KAP))[0]

# Generate data i.e. $\Delta$ vs $T_{\rm cry}$ for a list of $\nu_0$ and write in file

In [16]:
NNU0=10
NU0list=np.array([0.05+i*0.05 for i in range(0,NNU0)])
for j in range(0,NNU0):
    NU0=NU0list[j]
    for i in range(0,NGAP):
        TSClist[i]=sp.optimize.fsolve(gap_nth_SC_inv,0.1,args=(GAPlist[i],KAP,G0,GAMMA,GAMMAB*NU0,TCAV,NU0,EPS))[0]
        TCDWlist[i]=sp.optimize.fsolve(gap_nth_CDW_inv,0.1,args=(GAPlist[i],KAP,G0,GAMMA,GAMMAB*NU0,TCAV,NU0,EPS))[0]
    outfname=("./data/GapVsT_g%.2f_gam%.2f_gamB%.2f_Tcav%.2f_nu%.3f_smallgap.dat"%(G0,GAMMA,GAMMAB,TCAV,NU0))    
    file=open(outfname,"a")
    file.write("#GAP\t\tT_EQ\t\tT_SC\t\tT_CDW\n")
    for i in range(0,NGAP):
        file.write("%lf\t%lf\t%lf\t%lf\n" %(GAPlist[i],TEQlist[i],TSClist[i],TCDWlist[i]))
    file.close()

nu=0.050 start



/opt/miniconda3/lib/python3.13/site-packages/scipy/integrate/_quadpack_py.py:1260: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/var/folders/15/xk2stb214vn3c4rfd89q4sgr0000gn/T/ipykernel_33635/1613001264.py:9: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  TCDWlist[i]=sp.optimize.fsolve(gap_nth_CDW_inv,0.1,args=(GAPlist[i],KAP,G0,GAMMA,GAMMAB*NU0,TCAV,NU0,EPS))[0]
/var/folders/15/xk2stb214vn3c4rfd89q4sgr0000gn/T/ipykernel_33635/1613001264.py:9: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last five Jacobian evaluations.
  TCDWlist[i]=sp.optimize.fsolve(gap_nth_CDW_inv,0.1,args=(GAPlist[i],KAP,G0,GAMMA,GAMMAB*NU0,TCAV,NU0,EPS))[0]


nu=0.05 done

nu=0.100 start

nu=0.10 done

nu=0.150 start

nu=0.15 done

nu=0.200 start

nu=0.20 done

nu=0.250 start

nu=0.25 done

